<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **Using Containers with Estimators** </span>

In this notebook, we cover how to build a container, which must then be published to the Amazon Elastic Container Registry (ECR) to be used for our Estimator object. This notebook is intended to set up a container in the ECR to be used in the [sagemaker_studio/rapids_studio_hpo.ipynb](sagemaker_studio/rapids_studio_hpo.ipynb) example. 

<span style="color:#8735fb; font-size:22pt"> **Preamble** </span>

To get things rolling let's make sure we can query our AWS SageMaker execution role and session as well as our account ID and AWS region.

In [ ]:
import sagemaker
from helper_functions import *

In [ ]:
execution_role = sagemaker.get_execution_role()
session = sagemaker.Session()

account=!(aws sts get-caller-identity --query Account --output text)
region=!(aws configure get region)

In [ ]:
account, region

<span style="color:#8735fb; font-size:22pt"> **Key Choices** </span>

Let's go ahead and choose the configuration options for our HPO run. For more details about the choices being set, please refer to the [RAPIDS + SageMaker HPO extended notebook](rapids_sagemaker_hpo_extended.ipynb). All of these choices will be used to write our Dockerfile to create environment variables within our container; if you are using your own workflow and training scripts, be sure to write your Dockerfile accordingly. 

In [ ]:
# please choose dataset directory
dataset_directory = '3_year' # '1_year', '3_year', '10_year', 'NYC_taxi'

In [ ]:
# please choose learning algorithm
algorithm_choice = 'XGBoost'

assert (algorithm_choice in ['XGBoost', 'RandomForest', 'KMeans'])

In [ ]:
# please choose cross-validation folds
cv_folds = 3

assert (cv_folds >= 1)

In [ ]:
# please choose code variant
ml_workflow_choice = 'singleGPU' 

assert (ml_workflow_choice in ['singleCPU', 'singleGPU', 'multiCPU', 'multiGPU'])

<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **1. ML Workflow** </span>

<span style="color:#8735fb; font-size:20pt"> Python ML Workflow </span>

To run our training script or build a RAPIDS enabled SageMaker HPO, we first need to build a [SageMaker Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html), which to use a custom container requires a URI from the ECR. 

First, let's switch our working directory to the location of the Estimator entrypoint and library code.

In [ ]:
%cd code

In [ ]:
# %load train.py

In [ ]:
# %load workflows/MLWorkflowSingleGPU.py

<span style="display: block; text-align: center; color:#8735fb; font-size:30pt"> **2. Build Estimator** </span>

The SageMaker Estimator represents the containerized software stack that AWS SageMaker will replicate to each worker node.

The first step to building our Estimator, is to augment a RAPIDS container with our ML Workflow code from above, and push this image to Amazon Elastic Cloud Registry so it is available to SageMaker.


<span style="color:#8735fb; font-size:20pt"> 2.1 - Containerize and Push to ECR </span>

Now let's turn to building our container. Our container can either be built on top of the latest RAPIDS [ nightly ] image as a starting layer or the RAPIDS stable image.

In [ ]:
rapids_base_container = 'rapidsai/rapidsai-cloud-ml:latest'

Let's also decide on the full name of our container. This is the URI that will eventually be passed into our Estimator object. 

In [ ]:
image_base = 'cloud-ml-sagemaker'
image_tag  = rapids_base_container.split(':')[1]

In [ ]:
ecr_fullname = f"{account[0]}.dkr.ecr.{region[0]}.amazonaws.com/{image_base}:{image_tag}"

In [ ]:
ecr_fullname

<span style="color:#8735fb; font-size:18pt"> 2.1.1 - Write Dockerfile </span>

We write out the Dockerfile to disk, and in a few cells execute the docker build command. This Dockerfile is intended to be used with the [RAPIDS + SageMaker Studio HPO notebook](sagemaker_studio/rapids_studio_hpo.ipynb), however you may write your own Dockerfile to use your own container in a similar fashion. 


Let's now write our selected RAPIDS image layer as the first FROM statement in the the Dockerfile.

In [ ]:
with open('Dockerfile', 'w') as dockerfile: 
    dockerfile.writelines( f'FROM {rapids_base_container} \n\n'
                           f'ENV AWS_DATASET_DIRECTORY="{dataset_directory}"\n'
                           f'ENV AWS_ALGORITHM_CHOICE="{algorithm_choice}"\n'
                           f'ENV AWS_ML_WORKFLOW_CHOICE="{ml_workflow_choice}"\n'
                           f'ENV AWS_CV_FOLDS="{cv_folds}"\n')

Next let's append write the remaining pieces of the Dockerfile, namely adding the sagemaker-training-toolkit, flask, dask-ml, and copying our python code.

In [ ]:
%%writefile -a Dockerfile

# ensure printed output/log-messages retain correct order
ENV PYTHONUNBUFFERED=True

# path where SageMaker looks for code when container runs in the cloud
ENV CLOUD_PATH="/opt/ml/code"

# copy our latest [local] code into the container 
COPY . $CLOUD_PATH

# make the entrypoint script executable
RUN chmod +x $CLOUD_PATH/entrypoint.sh

WORKDIR $CLOUD_PATH
ENTRYPOINT ["./entrypoint.sh"]

Lastly, let's ensure that our Dockerfile correctly captured our base image selection.

In [ ]:
validate_dockerfile(rapids_base_container)
!cat Dockerfile

<span style="color:#8735fb; font-size:18pt"> 2.1.2 Build and Tag </span>

The build step will be dominated by the download of the RAPIDS image (base layer). If it's already been downloaded the build will take less than 1 minute.

In [ ]:
!docker pull $rapids_base_container

In [ ]:
%%time
!docker build . -t $ecr_fullname -f Dockerfile

<span style="color:#8735fb; font-size:18pt"> 2.1.3 - Publish to Elastic Cloud Registry (ECR) </span>

Now that we've built and tagged our container its time to push it to Amazon's container registry (ECR). Once in ECR, AWS SageMaker will be able to leverage our image to build Estimators and run experiments.


Docker Login to ECR

In [ ]:
docker_login_str = !(aws ecr get-login --region {region[0]} --no-include-email)

In [ ]:
!{docker_login_str[0]}

Create ECR repository [ if it doesn't already exist]

In [ ]:
repository_query = !(aws ecr describe-repositories --repository-names $image_base)
if repository_query[0] == '':
    !(aws ecr create-repository --repository-name $image_base)

Let's now actually push the container to ECR
> Note the first push to ECR may take some time (hopefully less than 10 minutes).

In [ ]:
!docker push $ecr_fullname

<span style="color:#8735fb; font-size:20pt"> 2.2 - Create Estimator </span>

Having built our container [ +custom logic] and pushed it to ECR, we can finally compile all of efforts into an Estimator instance in SageMaker Studio. Navigate to [sagemaker_studio/rapids_studio_hpo.ipynb](sagemaker_studio/rapids_studio_hpo.ipynb) to continue.

<span style="color:#8735fb; font-size:25pt"> **Rapids References** </span>

> [cloud-ml-examples](http://github.com/rapidsai/cloud-ml-examples)

> [RAPIDS HPO](https://rapids.ai/hpo)

> [cuML Documentation](https://docs.rapids.ai/api/cuml/stable/)

<span style="color:#8735fb; font-size:25pt"> **SageMaker References** </span>

> [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit)

> [Estimator Parameters](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

> Spot Instances [docs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html), and [blog]()